In [ ]:
import pandas as pd
import requests


df = pd.read_csv('Imdb_dataset.csv')

API_KEY = 'bf4cf184995f9d9c0fe2120c2f2113d2'
API_URL = "https://api.themoviedb.org/3/find/{}?external_source=imdb_id&api_key={}"

genre_counts = {'Romance': 0, 'Comedy': 0, 'Horror': 0, 'Drama': 0, 'Crime': 0, 'Action': 0, 'Adventure': 0,'Sci-Fi': 0,'Thriller': 0}

filtered_data = []

def get_poster_path_from_tmdb(imdb_id):
    url = API_URL.format(imdb_id, API_KEY)
    try:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            movie_results = data.get('movie_results', [])
            if movie_results:
                return movie_results[0].get("poster_path", '')
        return ''
    except Exception as e:
        print(f"Failed to fetch plot for {imdb_id}: {e}")
        return ''

temp=0
temp1=0

for _, row in df.iterrows():
    temp1+=1
    movie_id = row['movie_id']
    title = row['title']
    plot = row['plot']
    genres = [g.strip() for g in row['genres'].split('|') if g.strip()]
    pp = get_poster_path_from_tmdb(movie_id)

    if pp == None:
        continue

    filtered_data.append({
        'movie_id': movie_id,
        'title': title,
        'genres': '|'.join(genres),
        'img': pp,
        'plot':plot,
    })

    for g in genres:
        genre_counts[g] += 1

    temp+=1
    if temp%100==0:
        print(genre_counts,temp1,temp)

filtered_df = pd.DataFrame(filtered_data)

print("Final genre counts:")
print({k: v for k, v in genre_counts.items() if v > 0})


filtered_df.to_csv('filtered_imdb_with_plots_final.csv', index=False)


In [ ]:
import pandas as pd
import requests
import time
from collections import defaultdict

df = pd.read_csv('filtered_imdb_with_posters,csv')

API_KEY = 'bf4cf184995f9d9c0fe2120c2f2113d2'
API_URL = "https://api.themoviedb.org/3/find/{}?external_source=imdb_id&api_key={}"

genre_counts = {'Romance': 5000, 'Comedy': 5000, 'Drama': 5000, 'Horror': 5000, 'Crime': 5000, 'Action': 5000, 'Adventure': 4151, 'Thriller': 5000, 'Sci-Fi': 4568}
max_per_genre = 5000
filtered_data = []

def get_plot_from_tmdb(imdb_id):
    url = API_URL.format(imdb_id, API_KEY)
    try:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            movie_results = data.get('movie_results', [])
            if movie_results:
                return movie_results[0].get('overview', '')
        return ''
    except Exception as e:
        print(f"Failed to fetch plot for {imdb_id}: {e}")
        return ''

def can_add_row(genres, genre_counts):
    for g in genres:
        if genre_counts[g] >= max_per_genre:
            return False
    return True

temp = 0
for _, row in df.iterrows():
    movie_id = row['imdbId']
    title = row['title']
    genres = [g.strip() for g in row['genres'].split(',') if g.strip()]

    if not can_add_row(genres, genre_counts):
        continue

    pp = get_plot_from_tmdb(movie_id)
    if pp == '':
        continue

    filtered_data.append({
        'movie_id': movie_id,
        'title': title,
        'genres': '|'.join(genres),
        'plot': pp
    })

    for g in genres:
        genre_counts[g] += 1

    temp += 1
    if temp % 100 == 0:
        print(genre_counts)

    if all(v >= max_per_genre for v in genre_counts.values()):
        break

filtered_df = pd.DataFrame(filtered_data)

print("Final genre counts:")
print({k: v for k, v in genre_counts.items() if v > 0})

filtered_df.to_csv('filtered_imdb_with_plots_2.csv', index=False)
